# -- The producer in this file is using a differnt kafka Topic -- #

In [72]:
# 1. **Import** **libraries**
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [73]:
#Getting the path of the datasets
file_path = "./dataset/SWaT_Dataset_Attack_v0.csv"

In [74]:
dataset = pd.read_csv(file_path)

In [75]:
#Change 'A ttack' with Attack in the Attack dataframe

dataset['Normal/Attack'].unique()

dataset['Normal/Attack'].replace({'A ttack': 'Attack'}, inplace=True)

dataset['Normal/Attack'].unique()

array(['Normal', 'Attack'], dtype=object)

In [76]:
dataset.head(1)

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal


In [77]:
# Remove leading space from the 'Timestamp' column
dataset['Timestamp'] = dataset['Timestamp'].str.strip()

In [78]:
#convert timestamp to datatime format
dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp'], format= '%d/%m/%Y %I:%M:%S %p')



dataset.head()

dataset.shape

(449919, 53)

In [79]:
dataset.head(5)

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,2015-12-28 10:00:00,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,2015-12-28 10:00:01,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,2015-12-28 10:00:02,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,2015-12-28 10:00:03,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,2015-12-28 10:00:04,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal


In [80]:
df = dataset.set_index('Timestamp')

In [81]:
#Divide the data into two entities
#Feature Matrix
X = df.iloc[:, :-1]
#Dependent varable vector
Y = df.iloc[:, -1]

Y = Y.reset_index(drop=True)


In [82]:
X.head(2)


,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-12-28 10:00:00,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.307786,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1
2015-12-28 10:00:01,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.307786,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1


In [83]:

Y.head(2)


0    Normal
1    Normal
Name: Normal/Attack, dtype: object

In [84]:

X.info()
X.shape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 449919 entries, 2015-12-28 10:00:00 to 2016-01-02 14:59:59
Data columns (total 51 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   FIT101   449919 non-null  float64
 1   LIT101   449919 non-null  float64
 2   MV101    449919 non-null  int64  
 3   P101     449919 non-null  int64  
 4   P102     449919 non-null  int64  
 5   AIT201   449919 non-null  float64
 6   AIT202   449919 non-null  float64
 7   AIT203   449919 non-null  float64
 8   FIT201   449919 non-null  float64
 9   MV201    449919 non-null  int64  
 10  P201     449919 non-null  int64  
 11  P202     449919 non-null  int64  
 12  P203     449919 non-null  int64  
 13  P204     449919 non-null  int64  
 14  P205     449919 non-null  int64  
 15  P206     449919 non-null  int64  
 16  DPIT301  449919 non-null  float64
 17  FIT301   449919 non-null  float64
 18  LIT301   449919 non-null  float64
 19  MV301    449919 non-null  int64

(449919, 51)

In [85]:
# Divide features into numerical and categorical

num_col = []
cat_col = []

for column in X.columns:
    unique_values = X[column].nunique()
    if unique_values > 10:
        num_col.append(column)
    else:
        cat_col.append(column)

print('num column', num_col)
print('num column', cat_col)

print('num column', len(num_col))
print('num column', len(cat_col))

num column ['FIT101', 'LIT101', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'DPIT301', 'FIT301', 'LIT301', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'PIT501', 'PIT502', 'PIT503', 'FIT601']
num column ['MV101', 'P101', 'P102', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'P401', 'P402', 'P403', 'P404', 'UV401', 'P501', 'P502', 'P601', 'P602', 'P603']
num column 25
num column 26


In [86]:

numerical_data = X[num_col]
categorical_data = X[cat_col]

# Min-Max scaling for numerical features
scaler = MinMaxScaler()
numerical_data_scaled_ = scaler.fit_transform(numerical_data)
numerical_data_scaled_df_ = pd.DataFrame(numerical_data_scaled_, columns=numerical_data.columns)

# One-hot encoding for categorical features
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.fit(categorical_data)

categorical_feature_names = encoder.get_feature_names_out(categorical_data.columns)
categorical_data_encoded = encoder.transform(categorical_data)
categorical_data_encoded_df = pd.DataFrame(categorical_data_encoded, columns=categorical_feature_names)
preprocessed_data = pd.concat([numerical_data_scaled_df_, categorical_data_encoded_df], axis=1)



In [87]:
preprocessed_data.head(5)

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,MV303_1,MV303_2,MV304_1,MV304_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2
0,0.879322,0.452962,0.942783,0.876785,0.436773,0.865044,0.438853,0.928726,0.707350,0.5,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.886285,0.453015,0.942783,0.876785,0.436773,0.865044,0.438853,0.929319,0.707350,0.5,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
2,0.901833,0.452962,0.942783,0.876081,0.436773,0.863956,0.437572,0.929480,0.707733,0.5,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
3,0.918195,0.453122,0.942783,0.876081,0.436773,0.863956,0.437572,0.929480,0.708116,0.5,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
4,0.930842,0.453816,0.942783,0.876081,0.436773,0.864228,0.437572,0.929480,0.708451,0.5,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0


In [88]:

preprocessed_data.shape

(449919, 50)

In [89]:
Y = Y.apply(lambda x: 1 if x == 'Attack' else 0)

Y.head()


0    0
1    0
2    0
3    0
4    0
Name: Normal/Attack, dtype: int64

In [90]:
preprocessed_data['Normal/Attack'] = Y[0]

In [91]:
preprocessed_data.head(1)

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,MV303_2,MV304_1,MV304_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2,Normal/Attack
0,0.879322,0.452962,0.942783,0.876785,0.436773,0.865044,0.438853,0.928726,0.70735,0.5,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0


In [92]:
preprocessed_data.head()

,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,AIT401,...,MV303_2,MV304_1,MV304_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2,Normal/Attack
0,0.879322,0.452962,0.942783,0.876785,0.436773,0.865044,0.438853,0.928726,0.707350,0.5,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
1,0.886285,0.453015,0.942783,0.876785,0.436773,0.865044,0.438853,0.929319,0.707350,0.5,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
2,0.901833,0.452962,0.942783,0.876081,0.436773,0.863956,0.437572,0.929480,0.707733,0.5,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
3,0.918195,0.453122,0.942783,0.876081,0.436773,0.863956,0.437572,0.929480,0.708116,0.5,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
4,0.930842,0.453816,0.942783,0.876081,0.436773,0.864228,0.437572,0.929480,0.708451,0.5,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0


In [93]:
preprocessed_data = pd.concat([dataset["Timestamp"], preprocessed_data], axis=1)

In [94]:
preprocessed_data.head()

,Timestamp,FIT101,LIT101,AIT201,AIT202,AIT203,FIT201,DPIT301,FIT301,LIT301,...,MV303_2,MV304_1,MV304_2,P302_2,P402_2,P403_2,UV401_2,P501_2,P602_2,Normal/Attack
0,2015-12-28 10:00:00,0.879322,0.452962,0.942783,0.876785,0.436773,0.865044,0.438853,0.928726,0.707350,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
1,2015-12-28 10:00:01,0.886285,0.453015,0.942783,0.876785,0.436773,0.865044,0.438853,0.929319,0.707350,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
2,2015-12-28 10:00:02,0.901833,0.452962,0.942783,0.876081,0.436773,0.863956,0.437572,0.929480,0.707733,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
3,2015-12-28 10:00:03,0.918195,0.453122,0.942783,0.876081,0.436773,0.863956,0.437572,0.929480,0.708116,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0
4,2015-12-28 10:00:04,0.930842,0.453816,0.942783,0.876081,0.436773,0.864228,0.437572,0.929480,0.708451,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0


In [95]:
processed_data_path = "./dataset/processed_data.csv"
preprocessed_data.to_csv(processed_data_path, index=False)

Write to kafka Producer

In [66]:
from confluent_kafka import Producer
import json
import time
import pandas as pd

# Kafka configuration
kafka_config = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': 'python-producer'
}

# Kafka topic
topic = 'swat_topic_processed'

# Create Kafka producer
producer = Producer(kafka_config)

# Read from Pandas DataFrame and send to Kafka
i = 0
for _, row in preprocessed_data.iterrows():
    # Convert each row to JSON and send to Kafka
    message = json.dumps(row.to_dict(), default=str).encode('utf-8')
    producer.produce(topic, key='key', value=message)

    # Sleep 
    time.sleep(5)
    #  make sure to comment / uncomment this  stopping condition as per your need
    # i += 1
    # if i == 10:
    #     break

# Close the producer
producer.flush()


%4|1701820276.736|TERMINATE|python-producer#producer-2| [thrd:app]: Producer terminating with 78 messages (91426 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


0